# Bundle adjustment of $n$-cameras

### Goal

Suppose $n$ cameras have been calibrated linearly beforehand, using 2D-2D correspondences for example.  We assume we have an initial guess of $R$ and $t$ for each camera satisfying $x \sim A (R X + t)$, the 2D observation $x$, and the 3D point $X$.

* The initial guess is not necessarily done by linear methods.  It can even be done by hand.
* The 3D point $X$ can be triangulated from $x$ using the initial $R, t$.
* The $n$ cameras do not necessarily observe all the points.

Given $A, d, R, t, x, X$, this notebook optimizes $A, d, R, t, X$ so as to minimize the reprojection error.

* Input:
  * initial guess of $A, d, R, t, X$ and 2D observations $x$
* Output:
  * optimal $A, d, R, t, X$
 

## Libraries

In [1]:
%matplotlib notebook
import sys, os, cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from pycalib.plot import plotCamera
from pycalib.ba import Projection, Camera
from pycalib.calib import lookat, triangulate

## Synthetic data

In [6]:
# 3D points
# X_gt = (np.random.rand(16, 3) - 0.5)*5 # random points centered at [0, 0, 0]
X_gt = np.array(np.meshgrid(np.linspace(-1, 1, 3), np.linspace(-1, 1, 3), np.linspace(-1, 1, 3))).reshape((3, -1)).T  # 3D grid points
Np = X_gt.shape[0]
print('X_gt:', X_gt.shape)

# Camera intrinsics
K = np.array([[600, 0, 320], [0, 600, 240], [0, 0, 1]]).astype(np.float)  # VGA camera

# Camera poses: cameras are at the vertices of a hexagon
t = 2 * np.pi / 5 * np.arange(5)
v_gt = np.vstack((10*np.cos(t), 10*np.sin(t), np.zeros(t.shape))).T
Nc = v_gt.shape[0]
R_gt = []
t_gt = []
P_gt = []
rvec_gt = []
for i in range(Nc):
    t = v_gt[i,:]
    R, t = lookat(t, np.zeros(3), np.array([0, 1, 0]))
    R_gt.append(R)
    t_gt.append(t)
    P_gt.append(K @ np.hstack((R, t)))
    rvec_gt.append(cv2.Rodrigues(R)[0])
R_gt = np.array(R_gt)
t_gt = np.array(t_gt)
P_gt = np.array(P_gt)
rvec_gt = np.array(rvec_gt)
print('R_gt:', R_gt.shape)
print('t_gt:', t_gt.shape)
print('P_gt:', P_gt.shape)
print('rvec_gt:', rvec_gt.shape)

# 2D observations points
x_gt = []
for i in range(Nc):
    xt = cv2.projectPoints(X_gt.reshape((-1, 1, 3)), rvec_gt[i], t_gt[i], K, None)[0].reshape((-1, 2))
    x_gt.append(xt)
x_gt = np.array(x_gt)
print('x_gt:', x_gt.shape)

# Verify triangulation
Y = []
for i in range(Np):
    y = triangulate(x_gt[:,i,:].reshape((-1,2)), P_gt)
    #print(y)
    Y.append(y)
Y = np.array(Y).T
Y = Y[:3,:] / Y[3,:]
assert np.allclose(0, X_gt - Y.T)

# Verify z > 0 at each camera
for i in range(Nc):
    Xc = R_gt[i] @ X_gt.T + t_gt[i]
    assert np.all(Xc[2, :] > 0)

    
# Inject gaussian noise to the inital guess
R_est = R_gt.copy()
t_est = t_gt.copy()
K_est = np.array([K for c in range(Nc)])
X_est = X_gt.copy()
x_est = x_gt.copy()

for i in range(Nc):
    R_est[i] = cv2.Rodrigues( cv2.Rodrigues(R_est[i])[0] + np.random.normal(0, 0.01, (3,1)) )[0]
    t_est[i] += np.random.normal(0, 0.01, (3,1))
    K_est[i][0,0] = K_est[i][1,1] = K_est[i][0,0] + np.random.normal(0, K_est[i][0,0]/10)

X_est += np.random.normal(0, 0.01, X_est.shape)
x_est += np.random.normal(0, 0.1, x_est.shape)

X_gt: (27, 3)
R_gt: (5, 3, 3)
t_gt: (5, 3, 1)
P_gt: (5, 3, 4)
rvec_gt: (5, 3, 1)
x_gt: (5, 27, 2)


## Bundle adjustment

In [10]:
# build the model
cams = []
for i in range(Nc):
    cam = Camera(cv2.Rodrigues(R_est[i])[0], t_est[i].reshape(-1), K_est[i][0,0], None, K_est[i][0,2], K_est[i][1,2], np.zeros(5))
    cams.append(cam)

model = Projection(cams, X_est.T)

# Visibility mask: Nc x Np matrix
masks = torch.from_numpy(np.ones((Nc, Np)))

# 2D observations
pt2ds = torch.from_numpy(x_est.reshape(-1, 2))


device = torch.device('cpu')
model = model.to(device)
masks = masks.to(device)
pt2ds = pt2ds.to(device)

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
criterion = torch.nn.MSELoss()

model.train()
for i in range(1000):
    # project
    x = model.forward(masks)
    # reprojection error
    loss = criterion(x, pt2ds)
    
    if i % 100 == 0:
        print(f'E_rep[{i}] = {loss} px (mse)')

    # zero grad
    optimizer.zero_grad()
    # backward
    loss.backward()
    # update
    optimizer.step()

# final result
print(f'reprojection error = {loss} px')
print(model.cpu())

E_rep[0] = 51.52088008344347 px (mse)
E_rep[100] = 1.5773479049656325 px (mse)
E_rep[200] = 0.09931603319344978 px (mse)
E_rep[300] = 0.07340201157958046 px (mse)
E_rep[400] = 0.07236904182903235 px (mse)
E_rep[500] = 0.07197007143108793 px (mse)
E_rep[600] = 0.07171867233329984 px (mse)
E_rep[700] = 0.0715001816418563 px (mse)
E_rep[800] = 0.07129054638316439 px (mse)
E_rep[900] = 0.07108759943883594 px (mse)
reprojection error = 0.07089387474536153 px
Projection(
  (cameras): ModuleList(
    (0): Camera(
      rvec=[[0.09135562]
       [1.56258503]
       [0.09375596]],
      tvec=[ 1.94286244e-02 -8.30384620e-03  9.83404214e+00],
      fx=615.4560073930531, fy=615.4560073930531, cx=319.97653247454525, cy=239.91460846585994,
      dist=[0.018770119344296815, 0.04361984912285396, -0.0001725034144695716, -0.003732852211431912, 0.055973044451092725]
    )
    (1): Camera(
      rvec=[[-0.88273518]
       [ 1.38327359]
       [-0.87431132]],
      tvec=[2.02849118e-02 8.00760836e-04 1.06